In [1]:
import torch
#torch.set_num_threads(2)
import librosa
import whisper
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import japanize_matplotlib
#import evaluate
import gc
import spacy
import ginza
from tqdm.notebook import tqdm
import os
import math
from glob import glob
from torch.utils import data as data_utils

#GPUの利用
device = 0 if torch.cuda.is_available() else "cpu"
print(device)

num_data = 15000
#num_data = 100

C:\Users\toshi\anaconda3\envs\OpneCV_Face\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\toshi\anaconda3\envs\OpneCV_Face\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\toshi\anaconda3\envs\OpneCV_Face\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


cpu


In [2]:
# warning表示off
import warnings
warnings.simplefilter('ignore')

# デフォルトフォントサイズ変更
plt.rcParams['font.size'] = 14

# デフォルトグラフサイズ変更
plt.rcParams['figure.figsize'] = (6,6)

# デフォルトで方眼表示ON
plt.rcParams['axes.grid'] = True

# numpyの表示桁数設定
np.set_printoptions(suppress=True, precision=5)

In [3]:
#データの読み込み

metadata_csv_path = f'./datasets/fusion2/metadata.csv'

wavs = glob("./datasets/fusion2/wavs/*.wav", recursive=True)

fr = open( metadata_csv_path, "r", encoding='UTF-8')

datalist = fr.readlines()
audios = []
sentences = []
#filenames = []
i2 = 0
for i, line1 in enumerate(datalist):
    filename = line1.split( '|' )[0]
    #filename = filename.split('/')[-1]
    filename = "./datasets/fusion2/wavs\\" + filename + ".wav" #for windows
    #filename = "./datasets/fusion2/wavs/" + filename + ".wav"  #for Linux
    if filename in wavs:
        i2 += 1
        if i2 > num_data:
            break
        #filename = "./datasets/jsut_ver1.1/wavs/" + filename + ".wav"
        print( "i:{}, filename:{}".format( i, filename ))
        #音声ファイルの読み込み
        #filename = "./datasets/jsut_ver1.1/wavs/" + filename + ".wav"
        audio0, _ = librosa.load( filename, sr = 16000)
        #line2 = line1.split( '|' )[1]
        line2 = line1.split( '|' )[1].replace( "\n", "" )
        #line2 = line2.replace( "。", "" )
        #line2 = line2.replace( "、", "" )
        #print( "line2:{}".format(line2 ))
        #音声データ
        audios.append( audio0 )
        #正解文章データ
        sentences.append( line2 )
        #filenames.append( filename )
        
print( len( audios ))
print( len( sentences))

i:0, filename:./datasets/fusion2/wavs\BASIC5000_0001.wav
i:1, filename:./datasets/fusion2/wavs\BASIC5000_0002.wav
i:2, filename:./datasets/fusion2/wavs\BASIC5000_0003.wav
i:3, filename:./datasets/fusion2/wavs\BASIC5000_0004.wav
i:4, filename:./datasets/fusion2/wavs\BASIC5000_0005.wav
i:5, filename:./datasets/fusion2/wavs\BASIC5000_0006.wav
i:6, filename:./datasets/fusion2/wavs\BASIC5000_0007.wav
i:7, filename:./datasets/fusion2/wavs\BASIC5000_0008.wav
i:8, filename:./datasets/fusion2/wavs\BASIC5000_0009.wav
i:9, filename:./datasets/fusion2/wavs\BASIC5000_0010.wav
i:10, filename:./datasets/fusion2/wavs\BASIC5000_0011.wav
i:11, filename:./datasets/fusion2/wavs\BASIC5000_0012.wav
i:12, filename:./datasets/fusion2/wavs\BASIC5000_0013.wav
i:13, filename:./datasets/fusion2/wavs\BASIC5000_0014.wav
i:14, filename:./datasets/fusion2/wavs\BASIC5000_0015.wav
i:15, filename:./datasets/fusion2/wavs\BASIC5000_0016.wav
i:16, filename:./datasets/fusion2/wavs\BASIC5000_0017.wav
i:17, filename:./dataset

i:325, filename:./datasets/fusion2/wavs\BASIC5000_0326.wav
i:326, filename:./datasets/fusion2/wavs\BASIC5000_0327.wav
i:327, filename:./datasets/fusion2/wavs\BASIC5000_0328.wav
i:328, filename:./datasets/fusion2/wavs\BASIC5000_0329.wav
i:329, filename:./datasets/fusion2/wavs\BASIC5000_0330.wav
i:330, filename:./datasets/fusion2/wavs\BASIC5000_0331.wav
i:331, filename:./datasets/fusion2/wavs\BASIC5000_0332.wav
i:332, filename:./datasets/fusion2/wavs\BASIC5000_0333.wav
i:333, filename:./datasets/fusion2/wavs\BASIC5000_0334.wav
i:334, filename:./datasets/fusion2/wavs\BASIC5000_0335.wav
i:335, filename:./datasets/fusion2/wavs\BASIC5000_0336.wav
i:336, filename:./datasets/fusion2/wavs\BASIC5000_0337.wav
i:337, filename:./datasets/fusion2/wavs\BASIC5000_0338.wav
i:338, filename:./datasets/fusion2/wavs\BASIC5000_0339.wav
i:339, filename:./datasets/fusion2/wavs\BASIC5000_0340.wav
i:340, filename:./datasets/fusion2/wavs\BASIC5000_0341.wav
i:341, filename:./datasets/fusion2/wavs\BASIC5000_0342.w

i:519, filename:./datasets/fusion2/wavs\BASIC5000_0520.wav
i:520, filename:./datasets/fusion2/wavs\BASIC5000_0521.wav
i:521, filename:./datasets/fusion2/wavs\BASIC5000_0522.wav
i:522, filename:./datasets/fusion2/wavs\BASIC5000_0523.wav
i:523, filename:./datasets/fusion2/wavs\BASIC5000_0524.wav
i:524, filename:./datasets/fusion2/wavs\BASIC5000_0525.wav
i:525, filename:./datasets/fusion2/wavs\BASIC5000_0526.wav
i:526, filename:./datasets/fusion2/wavs\BASIC5000_0527.wav
i:527, filename:./datasets/fusion2/wavs\BASIC5000_0528.wav
i:528, filename:./datasets/fusion2/wavs\BASIC5000_0529.wav
i:529, filename:./datasets/fusion2/wavs\BASIC5000_0530.wav
i:530, filename:./datasets/fusion2/wavs\BASIC5000_0531.wav
i:531, filename:./datasets/fusion2/wavs\BASIC5000_0532.wav
i:532, filename:./datasets/fusion2/wavs\BASIC5000_0533.wav
i:533, filename:./datasets/fusion2/wavs\BASIC5000_0534.wav
i:534, filename:./datasets/fusion2/wavs\BASIC5000_0535.wav
i:535, filename:./datasets/fusion2/wavs\BASIC5000_0536.w

i:696, filename:./datasets/fusion2/wavs\BASIC5000_0697.wav
i:697, filename:./datasets/fusion2/wavs\BASIC5000_0698.wav
i:698, filename:./datasets/fusion2/wavs\BASIC5000_0699.wav
i:699, filename:./datasets/fusion2/wavs\BASIC5000_0700.wav
i:700, filename:./datasets/fusion2/wavs\BASIC5000_0701.wav
i:701, filename:./datasets/fusion2/wavs\BASIC5000_0702.wav
i:702, filename:./datasets/fusion2/wavs\BASIC5000_0703.wav
i:703, filename:./datasets/fusion2/wavs\BASIC5000_0704.wav
i:704, filename:./datasets/fusion2/wavs\BASIC5000_0705.wav
i:705, filename:./datasets/fusion2/wavs\BASIC5000_0706.wav
i:706, filename:./datasets/fusion2/wavs\BASIC5000_0707.wav
i:707, filename:./datasets/fusion2/wavs\BASIC5000_0708.wav
i:708, filename:./datasets/fusion2/wavs\BASIC5000_0709.wav
i:709, filename:./datasets/fusion2/wavs\BASIC5000_0710.wav
i:710, filename:./datasets/fusion2/wavs\BASIC5000_0711.wav
i:711, filename:./datasets/fusion2/wavs\BASIC5000_0712.wav
i:712, filename:./datasets/fusion2/wavs\BASIC5000_0713.w

i:886, filename:./datasets/fusion2/wavs\BASIC5000_0887.wav
i:887, filename:./datasets/fusion2/wavs\BASIC5000_0888.wav
i:888, filename:./datasets/fusion2/wavs\BASIC5000_0889.wav
i:889, filename:./datasets/fusion2/wavs\BASIC5000_0890.wav
i:890, filename:./datasets/fusion2/wavs\BASIC5000_0891.wav
i:891, filename:./datasets/fusion2/wavs\BASIC5000_0892.wav
i:892, filename:./datasets/fusion2/wavs\BASIC5000_0893.wav
i:893, filename:./datasets/fusion2/wavs\BASIC5000_0894.wav
i:894, filename:./datasets/fusion2/wavs\BASIC5000_0895.wav
i:895, filename:./datasets/fusion2/wavs\BASIC5000_0896.wav
i:896, filename:./datasets/fusion2/wavs\BASIC5000_0897.wav
i:897, filename:./datasets/fusion2/wavs\BASIC5000_0898.wav
i:898, filename:./datasets/fusion2/wavs\BASIC5000_0899.wav
i:899, filename:./datasets/fusion2/wavs\BASIC5000_0900.wav
i:900, filename:./datasets/fusion2/wavs\BASIC5000_0901.wav
i:901, filename:./datasets/fusion2/wavs\BASIC5000_0902.wav
i:902, filename:./datasets/fusion2/wavs\BASIC5000_0903.w

...

i:15294, filename:./datasets/fusion2/wavs\common_voice_ja_28274654.wav
i:15295, filename:./datasets/fusion2/wavs\common_voice_ja_28274655.wav
i:15296, filename:./datasets/fusion2/wavs\common_voice_ja_28274656.wav
i:15297, filename:./datasets/fusion2/wavs\common_voice_ja_28274657.wav
i:15298, filename:./datasets/fusion2/wavs\common_voice_ja_28274658.wav
i:15299, filename:./datasets/fusion2/wavs\common_voice_ja_28274731.wav
i:15300, filename:./datasets/fusion2/wavs\common_voice_ja_28274732.wav
i:15301, filename:./datasets/fusion2/wavs\common_voice_ja_28274733.wav
i:15302, filename:./datasets/fusion2/wavs\common_voice_ja_28274734.wav
i:15303, filename:./datasets/fusion2/wavs\common_voice_ja_28274735.wav
i:15304, filename:./datasets/fusion2/wavs\common_voice_ja_28274752.wav
i:15305, filename:./datasets/fusion2/wavs\common_voice_ja_28274753.wav
i:15306, filename:./datasets/fusion2/wavs\common_voice_ja_28274755.wav
i:15307, filename:./datasets/fusion2/wavs\common_voice_ja_28274756.wav
i:1530

i:15415, filename:./datasets/fusion2/wavs\common_voice_ja_30639864.wav
i:15416, filename:./datasets/fusion2/wavs\common_voice_ja_30639865.wav
i:15417, filename:./datasets/fusion2/wavs\common_voice_ja_30639871.wav
i:15418, filename:./datasets/fusion2/wavs\common_voice_ja_30639872.wav
i:15419, filename:./datasets/fusion2/wavs\common_voice_ja_30639873.wav
i:15420, filename:./datasets/fusion2/wavs\common_voice_ja_30639874.wav
i:15421, filename:./datasets/fusion2/wavs\common_voice_ja_30639875.wav
i:15422, filename:./datasets/fusion2/wavs\common_voice_ja_30639876.wav
i:15423, filename:./datasets/fusion2/wavs\common_voice_ja_30639877.wav
i:15424, filename:./datasets/fusion2/wavs\common_voice_ja_30639878.wav
i:15425, filename:./datasets/fusion2/wavs\common_voice_ja_30639879.wav
i:15426, filename:./datasets/fusion2/wavs\common_voice_ja_30639880.wav
i:15427, filename:./datasets/fusion2/wavs\common_voice_ja_30639886.wav
i:15428, filename:./datasets/fusion2/wavs\common_voice_ja_30639887.wav
i:1542

i:15606, filename:./datasets/fusion2/wavs\common_voice_ja_25124550.wav
i:15607, filename:./datasets/fusion2/wavs\common_voice_ja_25124551.wav
i:15608, filename:./datasets/fusion2/wavs\common_voice_ja_25124552.wav
i:15609, filename:./datasets/fusion2/wavs\common_voice_ja_25124553.wav
i:15610, filename:./datasets/fusion2/wavs\common_voice_ja_25124559.wav
i:15611, filename:./datasets/fusion2/wavs\common_voice_ja_25124560.wav
i:15612, filename:./datasets/fusion2/wavs\common_voice_ja_25124561.wav
i:15613, filename:./datasets/fusion2/wavs\common_voice_ja_25124562.wav
i:15614, filename:./datasets/fusion2/wavs\common_voice_ja_25124563.wav
i:15615, filename:./datasets/fusion2/wavs\common_voice_ja_25124583.wav
i:15616, filename:./datasets/fusion2/wavs\common_voice_ja_25124584.wav
i:15617, filename:./datasets/fusion2/wavs\common_voice_ja_25124585.wav
i:15618, filename:./datasets/fusion2/wavs\common_voice_ja_25124586.wav
i:15619, filename:./datasets/fusion2/wavs\common_voice_ja_25124587.wav
i:1562

i:15766, filename:./datasets/fusion2/wavs\common_voice_ja_21647032.wav
i:15767, filename:./datasets/fusion2/wavs\common_voice_ja_21647053.wav
i:15768, filename:./datasets/fusion2/wavs\common_voice_ja_21647055.wav
i:15769, filename:./datasets/fusion2/wavs\common_voice_ja_21647086.wav
i:15770, filename:./datasets/fusion2/wavs\common_voice_ja_21647087.wav
i:15771, filename:./datasets/fusion2/wavs\common_voice_ja_21647121.wav
i:15772, filename:./datasets/fusion2/wavs\common_voice_ja_21647122.wav
i:15773, filename:./datasets/fusion2/wavs\common_voice_ja_21647123.wav
i:15774, filename:./datasets/fusion2/wavs\common_voice_ja_21647125.wav
i:15775, filename:./datasets/fusion2/wavs\common_voice_ja_21666540.wav
i:15776, filename:./datasets/fusion2/wavs\common_voice_ja_21666692.wav
i:15777, filename:./datasets/fusion2/wavs\common_voice_ja_21666694.wav
i:15778, filename:./datasets/fusion2/wavs\common_voice_ja_21737162.wav
i:15779, filename:./datasets/fusion2/wavs\common_voice_ja_21737164.wav
i:1578

i:15898, filename:./datasets/fusion2/wavs\common_voice_ja_27983133.wav
i:15899, filename:./datasets/fusion2/wavs\common_voice_ja_27983134.wav
i:15900, filename:./datasets/fusion2/wavs\common_voice_ja_27983135.wav
i:15901, filename:./datasets/fusion2/wavs\common_voice_ja_27983198.wav
i:15902, filename:./datasets/fusion2/wavs\common_voice_ja_27983199.wav
i:15903, filename:./datasets/fusion2/wavs\common_voice_ja_27983200.wav
i:15904, filename:./datasets/fusion2/wavs\common_voice_ja_27983201.wav
i:15905, filename:./datasets/fusion2/wavs\common_voice_ja_27983202.wav
i:15906, filename:./datasets/fusion2/wavs\common_voice_ja_27983345.wav
i:15907, filename:./datasets/fusion2/wavs\common_voice_ja_27983346.wav
i:15908, filename:./datasets/fusion2/wavs\common_voice_ja_27983347.wav
i:15909, filename:./datasets/fusion2/wavs\common_voice_ja_27983348.wav
i:15910, filename:./datasets/fusion2/wavs\common_voice_ja_27983350.wav
i:15911, filename:./datasets/fusion2/wavs\common_voice_ja_27983381.wav
i:1591

In [4]:
#whisper モデルの読み込み
model = whisper.load_model( 'tiny' )

#tokenizer の読み込みと定数の設定
tokenizer = whisper.tokenizer.get_tokenizer(True, task="transcribe", language="ja")
eos = tokenizer.eot
special_tokens1 = list( tokenizer.sot_sequence_including_notimestamps )
#tokenizer.pad_token = eos
#tokenizer = whisper_tok.tokenizer
#eos = tokenizer.eot
#tokenizer.pad_token = eos
#special_tokens = tokenizer.sot_sequence_including_notimestamps

In [5]:
class Dataset_whisper(data_utils.Dataset):  # type: ignore
    """Dataset for numpy files

    Args:
        in_paths (list): List of paths to input files
        out_paths (list): List of paths to output files
    """

    def __init__(self, audio, sentence):
        self.audio = audio
        self.sentence = sentence

    def __getitem__(self, idx):
        """Get a pair of input and target

        Args:
            idx (int): index of the pair

        Returns:
            tuple: input and target in numpy format
        """
        #print( "idx:{}".format( idx ))
        #print( "self.x[idx]:{}".format( self.x[idx]))
        
        batch = dict()
        batch['audio'] = self.audio[idx]
        batch['sentence'] = self.sentence[idx]
        
        return batch

    def __len__(self):
        """Returns the size of the dataset

        Returns:
            int: size of the dataset
        """
        return len(self.audio)

In [6]:
def collate_fn_whisper(batch):
    """Collate function for Tacotron.

    Args:
        batch (list): List of tuples of the form (inputs, targets).
        reduction_factor (int, optional): Reduction factor. Defaults to 1.

    Returns:
        tuple: Batch of inputs, input lengths, targets, target lengths and stop flags.
    """
    #print( "batch:{}".format( batch ))
    
    #x = batch[0]['x']
    #y = batch[0]['y']
    
    #input_ids = torch.concat([to_pad_to_mel(x['audio'])[None, :] for x in batch])
    audios = [x['audio'] for x in batch]
    #tokenizer.pad(label_features, return_tensors="pt")
    labels2 = [ special_tokens1 + tokenizer.encode(y['sentence']) + [eos] for y in batch ]
    label2_lengths = [len(lab2) for lab2 in labels2 ]
    max_label2_len = max( label2_lengths )
    dec_input_ids = [np.pad(lab2, (0, max_label2_len - lab2_len ), 'constant', 
                     constant_values=eos) for lab2, lab2_len in zip(labels2, label2_lengths)]
    dec_input_ids = torch.tensor( np.array(dec_input_ids), requires_grad=False, dtype=torch.long)
    #dec_input_ids = [tokenizer.pad( y['sentence'], return_rensor="pt") for y in batch]
    #labels = [z['sentence'] for z in batch]
    #教師データ
    labels = [ special_tokens1[:1] + tokenizer.encode(z['sentence']) + [eos] for z in batch ]
    #デコーダーへ力
    label_lengths = [len(lab) for lab in labels]
    max_label_len = max(label_lengths)
    labels = [np.pad(lab, (0, max_label2_len - lab_len ), 'constant', 
              constant_values=-100) for lab, lab_len in zip(labels, label_lengths)]
    #教師データ
    labels = torch.tensor( np.array(labels), requires_grad=False, dtype=torch.long)
    
    return audios, dec_input_ids, labels

In [7]:
datasets = Dataset_whisper(audios, sentences)

train_ratio = 0.99
batch_size = 8

train_dataset, test_dataset = torch.utils.data.random_split(
    datasets, [int( len(datasets) * train_ratio), len(datasets) - int( len(datasets) * train_ratio)]
)

dataloader_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn_whisper, num_workers=0)
dataloader_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn_whisper, num_workers=0)

In [8]:
# モデルの重みとバイアス（パラメータ）の初期化

def kaiming_init(model):
    """ Kaimingの初期化

    Args:
        model (object): モデル
    """
    i = 1
    for name, param in model.named_parameters():
        print("i:{}".format( i ))
        i += 1
        if name.endswith(".bias"):
            print( "name:{}".format( name ))
            print( "shape of param:{}".format( param.shape ))
            print( "bias")
            param.data.fill_(0)
        elif name.startswith("encoder.conv"):
            print( "name:{}".format( name ))
            print( "shape of param:{}".format( param.shape ))
            print( "encoder.conv")
            param.data.normal_(0, 1/math.sqrt(param.shape[1]))
            #param.data.normal_(0, math.sqrt(2)/math.sqrt(param.shape[1]))
        elif name.endswith("ln.weight"):
            print( "name:{}".format( name ))
            print( "shape of param:{}".format( param.shape ))
            print("ln.weight")
            param.data.fill_(1)
        elif name.endswith("ln_post.weight"):
            print( "name:{}".format( name ))
            print( "shape of param:{}".format( param.shape ))
            print( "ln_post.weight")
            param.data.fill_(1)
        elif name.startswith("layers.0"):
             # The first layer does not have ReLU applied on its input
            print( "name:{}".format( name ))
            print( "shape of param:{}".format( param.shape ))
            print( "layers.0")
            param.data.normal_(0, 1/math.sqrt(param.shape[1]))
        elif name.endswith(".mlp.0.weight"):
            print( "name:{}".format( name ))
            print( "shape of param:{}".format( param.shape ))
            print(".mlp.0.weight" )
            param.data.normal_(0, math.sqrt(2)/math.sqrt(param.shape[1]))
        else:
            print( "name:{}".format( name ))
            print( "shape of param:{}".format( param.shape ))
            print("other")
            param.data.normal_(0, 1/math.sqrt(param.shape[1]))       
            
kaiming_init(model)

i:1
name:encoder.conv1.weight
shape of param:torch.Size([384, 80, 3])
encoder.conv
i:2
name:encoder.conv1.bias
shape of param:torch.Size([384])
bias
i:3
name:encoder.conv2.weight
shape of param:torch.Size([384, 384, 3])
encoder.conv
i:4
name:encoder.conv2.bias
shape of param:torch.Size([384])
bias
i:5
name:encoder.blocks.0.attn.query.weight
shape of param:torch.Size([384, 384])
other
i:6
name:encoder.blocks.0.attn.query.bias
shape of param:torch.Size([384])
bias
i:7
name:encoder.blocks.0.attn.key.weight
shape of param:torch.Size([384, 384])
other
i:8
name:encoder.blocks.0.attn.value.weight
shape of param:torch.Size([384, 384])
other
i:9
name:encoder.blocks.0.attn.value.bias
shape of param:torch.Size([384])
bias
i:10
name:encoder.blocks.0.attn.out.weight
shape of param:torch.Size([384, 384])
other
i:11
name:encoder.blocks.0.attn.out.bias
shape of param:torch.Size([384])
bias
i:12
name:encoder.blocks.0.attn_ln.weight
shape of param:torch.Size([384])
ln.weight
i:13
name:encoder.blocks.0.a

i:162
name:decoder.blocks.3.mlp.2.bias
shape of param:torch.Size([384])
bias
i:163
name:decoder.blocks.3.mlp_ln.weight
shape of param:torch.Size([384])
ln.weight
i:164
name:decoder.blocks.3.mlp_ln.bias
shape of param:torch.Size([384])
bias
i:165
name:decoder.ln.weight
shape of param:torch.Size([384])
ln.weight
i:166
name:decoder.ln.bias
shape of param:torch.Size([384])
bias


In [9]:
# ネットワークの概要表示

print(model)

Whisper(
  (encoder): AudioEncoder(
    (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
    (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
    (blocks): ModuleList(
      (0-3): 4 x ResidualAttentionBlock(
        (attn): MultiHeadAttention(
          (query): Linear(in_features=384, out_features=384, bias=True)
          (key): Linear(in_features=384, out_features=384, bias=False)
          (value): Linear(in_features=384, out_features=384, bias=True)
          (out): Linear(in_features=384, out_features=384, bias=True)
        )
        (attn_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=1536, out_features=384, bias=True)
        )
        (mlp_ln): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_post): LayerNorm((384,), eps=1e-05,

In [10]:
# GPUの利用
model = model

# 学習率
lr = 1e-2
#lr = 0.0004
#lr = 1e-1
#lr = 1.6e-5
#lr = 3.2e-6

# 損失関数定義
criterion = nn.CrossEntropyLoss(ignore_index=-100)

# エポック数
num_epochs1 = 100

# 最適化関数定義
#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr = lr )
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.0008, 
                                                steps_per_epoch=len(dataloader_train), epochs=num_epochs1)
lrs = []

# historyファイルと torch.save ファイルの path を初期化する
history = np.zeros((0, 8))
paths = []

In [11]:
#音声データ を 30秒に pad or trim して、mel フィルターバンクにする。
def to_pad_to_mel(array):
    """Static function which:
        1. Pads/trims a list of audio arrays to a max length of 30s
        2. Computes log-mel filter coefficients from padded/trimmed audio sequences
        Inputs:
            array: list of audio arrays
        Returns:
            input_ids: torch.tensor of log-mel filter bank coefficients
    """
    padded_input = whisper.pad_or_trim(np.asarray(array, dtype=np.float32))
    input_ids = whisper.log_mel_spectrogram(padded_input)
    return input_ids

In [12]:
# 学習用関数
def fit(model, optimizer, criterion, num_epochs, 
        dataloader_train, dataloader_test,
        device, lr, history, pahts):

    #torch.autograd.set_detect_anomaly(True)
    base_epochs = len(history)
    
    # cer, wer 計算ライブラリーの読み込み
    #metrics_cer = evaluate.load("cer")
    #metrics_wer = evaluate.load("wer")

    # wer 計算のために分かち書きをするので、分かち書き用。
    nlp = spacy.load("ja_ginza")
    ginza.set_split_mode(nlp, "C") # CはNEologdの意らしいです
    
    #メモリー確保
    gc.collect()
    
    epsilon = 1e-6
    
    # epoch のループ
    for epoch in range(base_epochs, num_epochs+base_epochs):
        train_loss = 0
        train_acc = 0
        val_loss = 0
        val_acc = 0
        #torch.save するファイルのファイル名。
        path = "whisper-torch-initialize28-tiny-" + (str(epoch+1)) + ".pth"
        #print( "path:{}".format(path))
        
        #訓練フェーズ
        model.train()
        count = 0
        step = 0
        
        #バッチのループ、プログレスバー対応　
        phar = tqdm( range( len(dataloader_train) ), desc='train' )
        Iter_train = iter(dataloader_train)
        for i in phar:
            step += 1 
            
            # ミニバッチのデータ
            audios, dec_input_ids, labels = next(Iter_train)
            input_ids = torch.concat([to_pad_to_mel(x)[None, :] for x in audios])
            
            gc.collect()
            #count += len(labels)
            count += 1
     
            gc.collect()
            
            # 勾配の初期化
            optimizer.zero_grad()
            
            # 予測計算
            logits = model(input_ids, tokens=dec_input_ids ) + epsilon
            
            # 損失計算
            loss = criterion( logits.view(-1, logits.size(-1)), labels.view(-1) )
           
            train_loss += loss.item()

            # 勾配計算
            loss.backward()

            # パラメータ修正
            optimizer.step()
            lrs.append(optimizer.param_groups[0]["lr"])
            current_lr = optimizer.param_groups[0]["lr"]
            #     print("Factor = ",i," , Learning Rate = ",optimizer.param_groups[0]["lr"])
            scheduler.step()

            # 予測値算出
            predicted = torch.max(logits, 2)[1]
    
            # 正解件数算出
            train_acc += (predicted == labels).sum()

            del logits
            del loss
            del predicted
            gc.collect()

            # 損失と精度の計算
            avg_train_loss = train_loss / count
            avg_train_acc = train_acc / count

            #プログラスバーに loss 表示
            phar.set_postfix( loss = avg_train_loss )            
        
        Iter_train2 = iter(dataloader_train)
        audios, dec_input_ids, labels = next(Iter_train2)
            
        label_str = []
        pred_str = []
        pred_str2 = []
        #for audio in audios:
        #    result = model.transcribe(audio, language="ja", task="transcribe")
        #    pred_str.append( result[0]['text'] )

        input_ids = torch.concat([to_pad_to_mel(x)[None, :] for x in audios])
        logits = model(input_ids, tokens=dec_input_ids ) + epsilon
        predicted = torch.max(logits, 2)[1]            

        labels[labels == -100] = eos
        predicted[predicted == -100] = eos        
        
        #for labels1, predicted2 in zip( labels, predicted1 ):
        min = np.min( [4, len(audios)])
        for i in range(min):
            #label_str.append( tokenizer.decode( labels1, skip_special_tokens=True ) )
            #pred_str.append( tokenizer.decode( predicted1, skip_special_tokens=True ) )
            label_str.append( tokenizer.decode( labels[i] ).replace( "<|endoftext|>", "" ).replace( "<|ja|>", "" )
                        .replace( "<|transcribe|>","").replace("<|startoftranscript|>","").replace("<|notimestamps|>",""))
            result = model.transcribe(audios[i], language="ja", task="transcribe")
            pred_str.append( result['text'] )
            pred_str2.append( tokenizer.decode( predicted[i] ).replace( "<|endoftext|>", "" ).replace( "<|ja|>", "" )
                        .replace( "<|transcribe|>","").replace("<|startoftranscript|>","").replace("<|notimestamps|>",""))    
                
        for j, _ in enumerate( label_str ):
            print( "train target:{}".format( label_str[j] ))
            print( "train predic:{}".format( pred_str[j] ))
            print( "train predi2:{}\n".format( pred_str2[j] ))

        
        #予測フェーズ
        model.eval()
        count = 0
        step = 0
        #cer_sum2 = 0
        #wer_sum2 = 0

        #バッチのループ、プログレスバー対応
        phar = tqdm( range( len(dataloader_test) ), desc='val' )
        Iter_test = iter(dataloader_test)
        for i in phar:
             # ミニバッチのデータ
            audios, dec_input_ids, labels = next(Iter_test)
            input_ids = torch.concat([to_pad_to_mel(x)[None, :] for x in audios])
            
            gc.collect()
            #count += len(labels)
            count += 1
            step += 1 

            gc.collect()
            
            # 予測計算
            logits = model(input_ids, tokens=dec_input_ids ) + epsilon
                
            # 損失計算
            loss = criterion( logits.view(-1, logits.size(-1)), labels.view(-1) )
            val_loss += loss.item()
 
            # 予測値算出
            predicted = torch.max(logits, 2)[1]

            # 正解件数算出
            val_acc += (predicted == labels).sum()

            labels[labels == -100] = eos
            predicted[predicted == -100] = eos

            #label_str = []
            #pred_str = []
            #for i in range( len(labels) ):
            #    #label_str.append( tokenizer.decode( labels1, skip_special_tokens=True ) )
            #    #pred_str.append( tokenizer.decode( predicted1, skip_special_tokens=True ) )
            #    label_str.append( tokenizer.decode( labels[i] ).replace( "<|endoftext|>", "" ).replace( "<|ja|>", "" )
            #            .replace( "<|transcribe|>","").replace("<|startoftranscript|>","").replace("<|notimestamps|>",""))
            #    #pred_str.append( tokenizer.decode( predicted[i] ).replace( "<|endoftext|>", "" ).replace( "<|ja|>", "" )
            #    #            .replace( "<|transcribe|>","").replace("<|startoftranscript|>","").replace("<|notimestamps|>",""))             
            
            # cer 算出
            #print( "label_str:{}".format( label_str ))
            #print( "pred_str:{}".format( pred_str ))
            #cer = 100 * metrics_cer.compute(predictions=pred_str, references=label_str)
            #cer_sum2 += cer

            # wer 算出
            # 分かち書きして空白区切りに変換
            #wer_pred_str = [" ".join([ str(i) for i in nlp(j) ]) for j in pred_str]
            #wer_label_str = [" ".join([ str(i) for i in nlp(j) ]) for j in label_str]            
            #wer = 100 * metrics_wer.compute(predictions=wer_pred_str, references=wer_label_str)                
            #wer_sum2 += wer                                 
            
            gc.collect()            
            
            # 損失と精度の計算ダミー
            avg_val_loss = val_loss / count
            avg_val_acc = val_acc / count
            # cer と wer の平均値計算。
            #avg_cer = cer_sum2 / step
            #avc_wer = wer_sum2 / step
            
            #プログラスバーに cer 表示
            phar.set_postfix( val_loss = avg_val_loss )   
        
        #avg_cer = cer_sum2 / (step )
        avg_cer = 0
        #print("avg_cer:{}".format( avg_cer ) )
        #avg_wer = wer_sum2 / (step )
        avg_wer = 0
        #print("avg_wer:{}".format( avg_wer ))
    
        Iter_test2 = iter(dataloader_test)
        audios, dec_input_ids, labels = next(Iter_test2)
            
        label_str = []
        pred_str = []
        pred_str2 = []
            
        #for audio in audios:
        #    result = model.transcribe(audio, language="ja", task="transcribe")
        #    pred_str.append( result[0]['text'] )

        input_ids = torch.concat([to_pad_to_mel(x)[None, :] for x in audios])
        logits = model(input_ids, tokens=dec_input_ids ) + epsilon
        predicted = torch.max(logits, 2)[1]            
            
        labels[labels == -100] = eos
        predicted[predicted == -100] = eos
            
            #for labels1, predicted2 in zip( labels, predicted1 ):
        min = np.min( [4, len(audios)])
        for i in range(min):
            #label_str.append( tokenizer.decode( labels1, skip_special_tokens=True ) )
            #pred_str.append( tokenizer.decode( predicted1, skip_special_tokens=True ) )
            label_str.append( tokenizer.decode( labels[i] ).replace( "<|endoftext|>", "" ).replace( "<|ja|>", "" )
                        .replace( "<|transcribe|>","").replace("<|startoftranscript|>","").replace("<|notimestamps|>",""))
            result = model.transcribe(audios[i], language="ja", task="transcribe")
            pred_str.append( result['text'] )
            pred_str2.append( tokenizer.decode( predicted[i] ).replace( "<|endoftext|>", "" ).replace( "<|ja|>", "" )
                        .replace( "<|transcribe|>","").replace("<|startoftranscript|>","").replace("<|notimestamps|>",""))    
                
        for j, _ in enumerate( label_str ):
            print( "val   target:{}".format( label_str[j] ))
            print( "val   predic:{}".format( pred_str[j] ))
            print( "val   predi2:{}\n".format( pred_str2[j] ))
            
    
        print (f'Epoch [{(epoch+1)}/{num_epochs+base_epochs}],loss: {avg_train_loss:.5f} acc: {avg_train_acc:.5f}, val-loss: {avg_val_loss:.5f} val-acc: {avg_val_acc:.5f}  cer: {avg_cer:.5f} wer: {avg_wer:.5f} lr: {current_lr:}')
        item = np.array([epoch+1, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc, avg_cer, avg_wer, current_lr])
        history = np.vstack((history, item))
        paths.append( path )
        
        # modelを torch.save する。
        #epoch が  1以上で前の epoch より val-los が小さければ save する。
        #if epoch >=1:
        #    if history[epoch,3] <= history[epoch-1,3]:
        #        torch.save( model, path )
        #        print( "model {} is saved.".format(path)) 
        #eoich が　0 の時は無条件で save
        #elif epoch == 0:
        #    torch.save( model, path )
        #    print( "model {} is saved.".format(path))

        # Early Stopping
        # 4 epoch 進んでも loss が改善しなかったら終了。
        if epoch >= 3:
            if history[epoch,1] >= history[epoch-3,1]:
                print( "Early Stopping.")
                break
        
    return history, pahts

In [13]:
#del audio_train
#del labels_train
#del audio_test
#del labels_test

gc.collect()

0

In [ ]:
# 学習
num_epochs = num_epochs1
history, paths = fit(model, optimizer, criterion, num_epochs, 
            dataloader_train, dataloader_test,
            device, lr, history, paths)

train:   0%|          | 0/1857 [00:00<?, ?it/s]

train target:賢明な判断が必要
train predic:がした�意�つじゃってる。たした
train predi2:このはははははにをを

train target:スイッチを切って音楽を止めます
train predic:し���に多なの
train predi2:あはははははは。

train target:ゼロ対ゼロの試合ばかりで観客も飽きてきた
train predic:なんか加かった�る���を落だ気し擝�
train predi2:こののははのは、に、にに、に、ににが。

train target:頑固なおじいさんだが、孫娘のカトリには甘い。
train predic:�学イ�サ、���と�のなるをににしいません
train predi2:�のの、、、、、、、、、、、に、、。を。。。。。。。



val:   0%|          | 0/19 [00:00<?, ?it/s]

val   target:彼とうまくやっていくことはできない。
val   predic:�いくと付以前らを�歌�め。
val   predi2:�の、は、、、、、、。。。。

val   target:あの、すみません。図書館はどこですか。
val   predic:中ージなわよでをにパみトうと�出候うきぶ
val   predi2:こののははの、、、、、に、を。。。。

val   target:けれども常識がいかに科学的になるにしても、
val   predic:からグ�றத학교�に、あ�吸も
val   predi2:こののははににににににを

val   target:醜いアヒルの子は、優雅な白鳥となった。
val   predic:ま�とかに、抽、死能入にも
val   predi2:�のは、、、、、、、、、に、、。。。

Epoch [1/100],loss: 5.99079 acc: 21.99300, val-loss: 5.50472 val-acc: 23.57895  cer: 0.00000 wer: 0.00000 lr: 3.410140464664153e-05


train:   0%|          | 0/1857 [00:00<?, ?it/s]

In [16]:
#記録する。
import pandas as pd

hist_df = pd.DataFrame(history)
hist_df.to_csv('kiroku_initialize33-tiny-15000-100.txt', header=False, index=False)
#hist_df.to_csv('test.csv')
'''
import numpy as np
history = np.loadtxt("kiroku_initialize33-tiny-15000-100.txt", delimiter=",", dtype = "unicode")

history = [[ float(s) for s in s1 ] for s1 in history]
#print(history_f)
# [-10, 0, 100]

print( "history", history )
'''

torch.save(model, 'model_weight_initialize33-tiny-15000-100.pth')

In [19]:


# 学習ログ解析
def evaluate_history(history):
    #損失と精度の確認
    print(f'初期状態: 損失: {history[0,3]:.5f} 精度: {history[0,4]:.5f} cer: {history[0,5]:.5f} wer: {history[0,6]:.5f}') 
    print(f'最終状態: 損失: {history[-1,3]:.5f} 精度: {history[-1,4]:.5f} cer: {history[-1,5]:.5f} wer: {history[-1,6]:.5f}' )

    num_epochs = len(history)
    if num_epochs < 10:
      unit = 1
    else:
      unit = num_epochs // 10

    # 学習曲線の表示 (損失)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,1], 'b', label='訓練')
    plt.plot(history[:,0], history[:,3], 'k', label='検証')
    plt.xticks(np.arange(0,num_epochs+1, unit))
    plt.xlabel('繰り返し回数')
    plt.ylabel('損失')
    plt.title('学習曲線(損失)')
    plt.legend()
    plt.show()

    # 学習曲線の表示 (精度)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,2], 'b', label='訓練')
    plt.plot(history[:,0], history[:,4], 'k', label='検証')
    plt.xticks(np.arange(0,num_epochs+1,unit))
    plt.xlabel('繰り返し回数')
    plt.ylabel('精度')
    plt.title('学習曲線(精度)')
    plt.legend()
    plt.show()
    
    # cer, wer
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,5], 'b', label='cer')
    plt.plot(history[:,0], history[:,6], 'k', label='wer')
    plt.xticks(np.arange(0,num_epochs+1,unit))
    plt.xlabel('繰り返し回数')
    plt.ylabel('cer,wer %')
    plt.title('cer,wer')
    plt.legend()
    plt.show()


In [20]:
# 結果サマリー
evaluate_history(history)

In [22]:
#result = model.transcribe("test-onsei/20230101121239.wav", language="ja", task="transcribe")
result = model.transcribe("P/data/p0001.wav", language="ja", task="transcribe")

print( result["text"] )

甡内の仲内動黸から江神分日には師圧盇一 Guys視妹景逌教容紀議害形軲み用に底でも国父�に功設作に心音中事を耐正期揞員やし漢街しになり相用力�網自習水別実水奦集決政治基本定志限競して知り�本当ników任��留し市解数攻して混運理種�有期集使気議多認法刹します没定生華生中�並十心�節行�運一し�少定流率休心め�的 firsthand用竳もに慢して多もれない死�石投設定的特談�当�事��親用作当人で幸シしか定�費�龲会低�的�の給し性の�り大��業み水�さ分�国種さ設国
